# Stats
## Peak freq

In [ ]:
import numpy as np
from sklearn.utils import resample
import scipy.signal as ss
import mne

# for plotting, remove later
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
EEG_h = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_h.npy',allow_pickle=True).item(0)

In [ ]:

peaks = []
for i in range(60):
    peakind = np.where(EEG_h[str(1000+i)]['ps_wel']==max(EEG_h[str(1000+i)]['ps_wel']))[0][0]
    peaks.append(EEG_h['mean_psd']['freq_wel'][peakind])

peaks = np.array(peaks)

print(f'Max peak = {round(peaks.mean(),2)} +/- {round(peaks.std(),2)} Hz')

## Spectrogram events

### Elevated power 1 sd larger than signal mean

In [ ]:
def bandPassFilter(signal,low,high):
    order = 2
    b, a = ss.butter(order,[low,high],btype='bandpass',fs=sampling_rate)
    y = ss.filtfilt(b, a, signal)
    return(y)

In [ ]:
low, hight = 4,12
sampling_rate = 40000
pwidth = 40000*.100 # event must last for 100ms

plot = False

w = []
n = []

for i in range(60):
    bp = bandPassFilter(EEG_h[str(1000+i)]['ts_raw'][80000:],low,hight)
    detrend_env = ss.detrend(np.abs(ss.hilbert((bp))),type='constant')
    thresh = detrend_env.std()
    p = ss.find_peaks(detrend_env,height=thresh,width=pwidth)
    
    w.append(p[1]['widths'].mean()/40000)
    n.append(len(p[1]['widths']))
    

    if plot and i==0:
        fig = plt.figure(figsize=(15,5))
        ax1 = plt.subplot(111)
        ax1.plot(detrend_env)
        ax1.axhline(thresh,color='k')
        ax1.scatter(p[0],p[1]['peak_heights'],color='orange')
        ax1.set_title('Theta-Alpha Events')

        
w = np.array(w)
n = np.array(n)

print(f'{round(n.mean(),2)} +/- {round(n.std(),2)} theta-alpha events,lasting {round(w.mean(),3)} +/- {round(w.std(),3)} s')
 